# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%pip install causal_conv1d

In [ ]:
os.environ['MAX_JOBS'] = '3'
%pip install ninja
%pip install --no-build-isolation --verbose ./selective_scan

In [3]:
import torch, InitMamba
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

# Lab

In [4]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
inputs = tokenizer(["Today is Monday, Tomorrow is Tuesday.",
           "Today is Sunday, Tomorrow is Monday.",
           "Today is Sunday, Tomorrow is Monday."],
          #   max_length = 32,
          #  padding = 'max_length',
           return_tensors='pt').to('cuda')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
model1.train()

res1 = model1(**inputs, output_ssm_last_states = True)
ssm_last_states = res1.ssm_last_states
print(ssm_last_states[-1])

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


tensor([[ 2.4119e-02, -9.5605e-02,  6.1495e-03,  ...,  7.5622e-02,
         -1.4609e-01, -1.8936e-01],
        [ 8.2959e-03, -1.2948e-02,  1.5957e-04,  ..., -3.7550e-03,
          6.0886e-03,  9.3989e-03],
        [-7.3239e-02,  8.2168e-02, -3.4553e-03,  ..., -3.9539e-02,
          8.6269e-02,  1.0851e-01],
        ...,
        [ 9.7279e-03, -1.3224e-02,  4.3392e-03,  ...,  3.3171e-02,
         -6.1610e-02, -8.0464e-02],
        [-1.2079e-01,  6.0100e-02, -1.0434e-02,  ..., -1.1579e-01,
          2.1582e-01,  2.8984e-01],
        [-6.4843e-03,  9.4886e-03, -1.6532e-03,  ..., -1.3224e-02,
          3.7167e-02,  5.0888e-02]], device='cuda:0',
       grad_fn=<SelectBackward0>)


In [9]:
res = model1(**inputs, output_ssm_last_states = True, inputs_ssm_states = ssm_last_states)
print(res.logits)
tokenizer.batch_decode(res.logits.argmax(-1))

tensor([[[  9.9198,   1.5491,   2.6419,  ...,   1.6821,   1.6572,   1.5468],
         [  4.7165, -12.5163,   3.3999,  ..., -12.7500, -12.6262, -12.6619],
         [-19.7026, -46.9598, -22.1777,  ..., -46.7554, -46.8826, -46.8495],
         ...,
         [ -9.2194,   7.6863, -16.2589,  ...,   7.3706,   7.2829,   7.6166],
         [-13.6607, -18.8684, -18.2374,  ..., -18.5683, -18.7283, -18.9281],
         [  3.5942, -16.5568,   2.2905,  ..., -16.8151, -17.3365, -16.8327]],

        [[ 19.3583,  -0.6694,  13.4369,  ...,  -0.8893,  -0.7612,  -0.9876],
         [-11.5463, -25.2096, -10.4256,  ..., -25.2801, -25.2087, -25.3942],
         [ -0.7773, -17.1977,  -6.9741,  ..., -17.1582, -17.1418, -17.0683],
         ...,
         [ -2.7754,  -6.5402, -10.4917,  ...,  -6.7587,  -7.0094,  -6.6512],
         [ 10.1860, -15.5430,  -6.6677,  ..., -15.2001, -15.6132, -15.4658],
         [  7.5376, -31.5117,   2.6149,  ..., -31.8905, -32.1699, -31.5726]],

        [[ 19.3583,  -0.6694,  13.4369,  ...

['ethod and firstloatenselyariju our d',
 'namese and theirante Sbben T d',
 'namese and theirante Sbben T d']

In [8]:
torch.autograd.grad(outputs=res.logits.mean(), inputs=ssm_last_states)

(tensor([[[ 0.2946,  1.0394,  0.3819,  ...,  1.8289, -0.8378,  0.7556],
          [-3.9220,  0.2882,  0.6721,  ...,  0.0723,  2.0089, -0.6532],
          [-0.4168, -0.7216,  0.0279,  ..., -0.8134,  0.4345,  0.0042],
          ...,
          [-0.0890,  0.5959, -0.7744,  ..., -1.4096, -0.1136, -1.0288],
          [ 0.5794,  0.3463,  0.4186,  ..., -0.6908,  0.4392,  0.0123],
          [-0.6525, -1.0777,  3.2442,  ...,  0.8828,  2.3689,  0.8545]],
 
         [[-0.0501,  1.0625,  0.1401,  ...,  0.2090,  0.0199, -0.0977],
          [-1.2707,  0.3201,  0.0247,  ...,  0.0203,  0.7554, -0.2175],
          [-0.0379, -0.1715,  0.0076,  ..., -0.1561, -0.4653,  0.0226],
          ...,
          [ 0.3033,  0.0045, -0.0945,  ...,  0.0733, -0.3215,  0.1298],
          [-0.0877, -0.0674, -0.0232,  ..., -0.0895,  0.1850,  0.2670],
          [-0.2333, -0.1446,  0.8714,  ...,  0.3004,  0.5954,  0.1813]],
 
         [[-0.0501,  1.0625,  0.1401,  ...,  0.2090,  0.0199, -0.0977],
          [-1.2707,  0.3201,

In [ ]:
from google.colab import runtime
runtime.unassign()